In [1]:
! pip3 install wldhx.yadisk-direct

In [2]:
! curl -L $(yadisk-direct https://disk.yandex.ru/d/BYmUMhZtFjjJRA) -o data.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 1034M  100 1034M    0     0  9501k      0  0:01:51  0:01:51 --:--:-- 10.0M


In [3]:
import torch

In [4]:
data = torch.load('data.pt')

In [5]:
features = data.keys()
features

dict_keys(['BannerTsarVector13', 'BannerTsarVector14', 'BannerTsarVector18', 'BannerTsarVector21', 'BannerTsarVector22', 'BannerTsarVector23', 'BannerTsarVector24', 'BannerTsarVector25', 'BannerTsarVector3', 'BannerTsarVector31', 'BannerTsarVector36', 'BannerTsarVector7', 'HitTsarVector13', 'HitTsarVector14', 'HitTsarVector18', 'HitTsarVector21', 'HitTsarVector22', 'HitTsarVector23', 'HitTsarVector24', 'HitTsarVector25', 'HitTsarVector3', 'HitTsarVector31', 'HitTsarVector36', 'HitTsarVector7', 'TsarVector13', 'TsarVector16', 'TsarVector18', 'TsarVector21', 'TsarVector23', 'TsarVector24', 'TsarVector26', 'TsarVector27', 'TsarVector3', 'TsarVector31', 'TsarVector4', 'Ctr', 'BidCorrection', 'ABConversionCostCoef', 'Bid', 'HitLogID', 'BannerID'])

In [6]:
data

{'ABConversionCostCoef': tensor([0.1007, 0.0467, 1.0000,  ..., 0.0915, 0.0684, 0.2100]),
 'BannerID': tensor([72057605009416680, 72057604936041032, 72057604702625217,
          ..., 72057604631764122, 72057604148230380,
         72057604184459582]),
 'BannerTsarVector13': tensor([[-0.0451, -0.0069,  0.0311,  ...,  0.0295, -0.0136, -0.0154],
         [-0.0105, -0.0151, -0.0150,  ...,  0.0266, -0.0027,  0.0062],
         [-0.0061, -0.0110,  0.0340,  ...,  0.0167, -0.0053, -0.0279],
         ...,
         [ 0.0641, -0.0194,  0.0279,  ...,  0.0181, -0.0172, -0.0089],
         [ 0.0563, -0.0212,  0.0162,  ..., -0.0047, -0.0015, -0.0333],
         [ 0.0790, -0.0166,  0.0255,  ...,  0.0104, -0.0093, -0.0193]]),
 'BannerTsarVector14': tensor([[-0.0964, -0.0407,  0.1269,  ...,  0.1180,  0.0692,  0.2106],
         [-0.1387,  0.0740,  0.0822,  ...,  0.0396,  0.0413,  0.2540],
         [-0.0946,  0.0227,  0.0656,  ...,  0.1869,  0.0371,  0.2071],
         ...,
         [-0.1454, -0.1178,  0.1082, 

In [7]:
BANNER_ID = "BannerID"
HIT_LOG_ID = "HitLogID"
BANNER_FEATURES = ["BannerTsarVector13", "BannerTsarVector14", "BannerTsarVector18", "BannerTsarVector21", "BannerTsarVector22", "BannerTsarVector23", "BannerTsarVector24", "BannerTsarVector25", "BannerTsarVector3", "BannerTsarVector31", "BannerTsarVector36", "BannerTsarVector7"]
HIT_FEATURES = ["HitTsarVector13", "HitTsarVector14", "HitTsarVector18", "HitTsarVector21", "HitTsarVector22", "HitTsarVector23", "HitTsarVector24", "HitTsarVector25", "HitTsarVector3", "HitTsarVector31", "HitTsarVector36", "HitTsarVector7", "TsarVector13", "TsarVector16", "TsarVector18", "TsarVector23", "TsarVector24", "TsarVector26", "TsarVector27", "TsarVector3", "TsarVector31", "TsarVector4"]
TARGETS = ["Ctr", "BidCorrection", "ABConversionCostCoef"]

https://pytorch.org/tutorials/advanced/cpp_export.html

In [18]:
class DummyModel:
    def __init__(self):
        pass

    def __call__(self, x):
        return torch.randn(x[BANNER_FEATURES[0]].shape[0])

In [19]:
model = DummyModel()

In [20]:
def group_data(data):
    grouped_data = defaultdict(list)
    for i, x in enumerate(data['HitLogID'].numpy().tolist()):
        grouped_data[x].append(i)
    
    for group in grouped_data.values():
        batch = {
            key: val[group]
            for key, val in data.items()
        }
        yield batch


def eval_metric(model, data):
    metrics = []
    for batch in group_data(data):
        with torch.no_grad():
            target = sum(batch[t].log() for t in TARGETS)

            expected_order = model(batch).argsort().numpy().tolist()
            real_order = target.argsort().numpy().tolist()

            top_n = len(expected_order) // 4
            # print(, len(expected_order))
            metrics.append(len(set(expected_order[-top_n:]).intersection(set(real_order[-top_n:]))) / top_n)
    return sum(metrics) / len(metrics)

In [21]:
eval_metric(model, data)

0.2549869427345646